In [ ]:
!pip install controlnet_aux diffusers gradio mediapipe transformers

In [ ]:
import gradio as gr

import torch

from controlnet_aux.processor import Processor

from diffusers import AutoPipelineForInpainting, ControlNetModel
from diffusers import DPMSolverMultistepScheduler, EulerDiscreteScheduler, UniPCMultistepScheduler
from diffusers import StableDiffusionControlNetPipeline as SDCNPipeline

In [ ]:
canny = Processor("canny")

controlnet = ControlNetModel.from_pretrained(
  "lllyasviel/sd-controlnet-canny",
  torch_dtype=torch.float16
)

pipe = SDCNPipeline.from_pretrained(
  "runwayml/stable-diffusion-v1-5",
  controlnet=controlnet,
  safety_checker=None,
  torch_dtype=torch.float16
).to("cuda")

pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

In [ ]:
def gen_image(img, txt):
  pimg = canny(img, to_pil=True)
  res = pipe(txt, pimg, num_inference_steps=20)
  return res.images[0]

with gr.Blocks() as demo:
  gr.Interface(
    gen_image,
    inputs=[gr.Image(type="pil"), gr.Textbox()],
    outputs="image",
  )

  demo.launch()